In [1]:
def populatebycolumn(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                names=my_rownames)

    c = [[[0, 1], [-1.0, 1.0]],
         [[0, 1], [1.0, -3.0]],
         [[0, 1], [1.0, 1.0]]]

    prob.variables.add(obj=my_obj, ub=my_ub, names=my_colnames,
                       columns=c)

In [2]:
def print_lp_result(my_prob):
    for i, x in enumerate(my_prob.solution.get_values()):
        print(my_prob.variables.get_names(i), x)
    print ("Solution value:   ",  my_prob.solution.get_objective_value())

Maximize
x1  + 2x2 + 3x3
subject to
–x1 +  x2 + x3 <= 20
 x1 – 3x2 + x3 <= 30
with these bounds
0 <= x1 <= 40
0 <= x2 <= infinity
0 <= x3 <= infinity

In [3]:
import cplex
from cplex.exceptions import CplexError

# data common to all populateby functions
my_obj = [1.0, 2.0, 3.0]
my_ub = [40.0, cplex.infinity, cplex.infinity]
my_colnames = ["x1", "x2", "x3"]
my_rhs = [20.0, 30.0]
my_rownames = ["c1", "c2"]
my_sense = "LL"

my_prob = cplex.Cplex()
populatebycolumn(my_prob)
my_prob.solve()
print_lp_result()
my_prob.write("lpex1.lp")

CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
No LP presolve or aggregator reductions.
Presolve time = 0.00 sec. (0.00 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =             0.000000
Iteration:     2   Dual objective     =           202.500000


TypeError: print_lp_result() missing 1 required positional argument: 'my_prob'

my_prob.write("lpex1.lp") 可以把模型输出到 文件lpex1.lp
下次需要求解同样的模型的时候 直接采用new_my_prob.read 可以直接导入模型

In [4]:
new_my_prob = cplex.Cplex()
new_my_prob.read('lpex1.lp')
new_my_prob.solve()

CPLEX Error  1423: Could not open file 'lpex1.lp' for reading.
CPLEX Error  1423: Could not open file 'lpex1.lp.sav' for reading.
CPLEX Error  1423: Could not open file 'lpex1.lp' for reading.
CPLEX Error  1423: Could not open file 'lpex1.lp' for reading.


CplexSolverError: CPLEX Error  1423: Could not open file 'lpex1.lp' for reading.

In [ ]:
def populatebyrow(prob):
    prob.objective.set_sense(prob.objective.sense.maximize)

    # since lower bounds are all 0.0 (the default), lb is omitted here
    prob.variables.add(obj = my_obj, ub = my_ub, names = my_colnames)

    # can query variables like the following:

    # lbs is a list of all the lower bounds
    lbs = prob.variables.get_lower_bounds()

    # ub1 is just the first lower bound
    ub1 = prob.variables.get_upper_bounds(0) 

    # names is ["x1", "x3"]
    names = prob.variables.get_names([0, 2])

    rows = [[["x1","x2","x3"],[-1.0, 1.0,1.0]],
            [["x1","x2","x3"],[ 1.0,-3.0,1.0]]]

    prob.linear_constraints.add(lin_expr = rows, senses = my_sense,
                                rhs = my_rhs, names = my_rownames)

    # because there are two arguments, they are taken to specify a range
    # thus, cols is the entire constraint matrix as a list of column vectors
    cols = prob.variables.get_cols("x1", "x3")
    alg = prob.parameters.lpmethod.values
    prob.parameters.lpmethod.set(alg.auto)

In [ ]:
import cplex
from cplex.exceptions import CplexError

# data common to all populateby functions
my_obj = [1.0, 2.0, 3.0]
my_ub = [40.0, cplex.infinity, cplex.infinity]
my_colnames = ["x1", "x2", "x3"]
my_rhs = [20.0, 30.0]
my_rownames = ["c1", "c2"]
my_sense = "LL"

my_prob_row = cplex.Cplex()

#alg = my_prob_row.parameters.lpmethod.values
#my_prob_row.parameters.lpmethod.set(alg.primal)

populatebyrow(my_prob_row)
my_prob_row.solve()
print_lp_result(my_prob_row)